In [5]:
%pip install pypdf tokenizers==0.19.1 unstructured pdf2image pdfminer.six opencv-python pytesseract unstructured_inference pikepdf python-dotenv 
%pip install langchain langchain_community langchain_mistralai langchain_anthropic langchain_openai

  Using cached tokenizers-0.19.0-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached tokenizers-0.19.0-cp312-cp312-macosx_11_0_arm64.whl (2.4 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-mistralai 0.1.6 requires tokenizers<0.16.0,>=0.15.1, but you have tokenizers 0.19.0 which is incompatible.
Note: you may need to restart the kernel to use updated packages.
  Using cached tokenizers-0.15.2-cp312-cp312-macosx_11_0_arm64.whl.metadata (6.7 kB)
Using cached tokenizers-0.15.2-cp312-cp312-macosx_11_0_arm64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.0
    Uninstalling tokenizers-0.19.0:
      Successfully uninstalled tokenizers-0.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following 

# Ingest Data

 Get text from pdf file

In [1]:
# for pdf documents
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("law.pdf")
documents_simple = loader.load()
documents_simple

[Document(page_content='Закон РФ от 7 февраля 1992 г. N 2300 -I "О защите прав потребителей" (с изменениями и дополнениями)  \n29.06.2021   Система ГАРАНТ  1/27 \n \nЗАКОН  \n \nРОССИЙСКОЙ ФЕДЕРАЦИИ \n \n \nО защите прав потребителей  \n(в редакции  Федерального закона от 9 января 1996 года N 2- ФЗ) \n(с изменениями на 11 июня 2021 года)  \n(редакция, действующая с 1 июля 2021 года)  \nИнформация об изменяющих документах  \n \nНастоящий Закон в редакции  Федерального закон а от 9 января 1996 года N \n2-ФЗ введен в действие с 15 января 1996 года, кроме положений, содержащихся в пунктах \n2 и 4  статьи 5 , абзаце первом пункта 4  статьи 7  и в абзаце восьмом пункта 1  статьи 18 . \nПоложения, содержащиеся в перечисленных пунктах, вступают в силу со дня утверждения Правительством Российской Фе дерации соответствующих перечней - см. статью 3 \nФедерального закона от 9 января 1996 года N 2- ФЗ. \n- Примечание изготовителя базы данных.  \n_____________________________________________________

In [7]:
# better pdf document loader
from langchain_community.document_loaders import UnstructuredPDFLoader
loader = UnstructuredPDFLoader("law.pdf")
documents = loader.load()
documents[0].page_content

'Закон РФ от 7 февраля 1992 г. N 2300-I "О защите прав потребителей" (с изменениями и дополнениями)\n\nЗАКОН\n\nРОССИЙСКОЙ ФЕДЕРАЦИИ\n\nО защите прав потребителей\n\n(в редакции Федерального закона от 9 января 1996 года N 2-ФЗ) (с изменениями на 11 июня 2021 года) (редакция, действующая с 1 июля 2021 года)\n\nИнформация об изменяющих документах\n\nНастоящий Закон в редакции Федерального закона от 9 января 1996 года N\n\n2-ФЗ введен в действие с 15 января 1996 года, кроме положений, содержащихся в пунктах 2 и 4 статьи 5, абзаце первом пункта 4 статьи 7 и в абзаце восьмом пункта 1 статьи 18. Положения, содержащиеся в перечисленных пунктах, вступают в силу со дня утверждения Правительством Российской Федерации соответствующих перечней - см. статью 3 Федерального закона от 9 января 1996 года N 2-ФЗ. - Примечание изготовителя базы данных.\n\n____________________________________________________________________\n\nНастоящий Закон регулирует отношения, возникающие между потребителями и изготов

 Split text

In [8]:
# simple recursive text splitter with overlap
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunked_document=text_splitter.split_documents(documents=documents)
print(chunked_document)
print(len(chunked_document))

[Document(page_content='Закон РФ от 7 февраля 1992 г. N 2300-I "О защите прав потребителей" (с изменениями и дополнениями)\n\nЗАКОН\n\nРОССИЙСКОЙ ФЕДЕРАЦИИ\n\nО защите прав потребителей\n\n(в редакции Федерального закона от 9 января 1996 года N 2-ФЗ) (с изменениями на 11 июня 2021 года) (редакция, действующая с 1 июля 2021 года)\n\nИнформация об изменяющих документах\n\nНастоящий Закон в редакции Федерального закона от 9 января 1996 года N\n\n2-ФЗ введен в действие с 15 января 1996 года, кроме положений, содержащихся в пунктах 2 и 4 статьи 5, абзаце первом пункта 4 статьи 7 и в абзаце восьмом пункта 1 статьи 18. Положения, содержащиеся в перечисленных пунктах, вступают в силу со дня утверждения Правительством Российской Федерации соответствующих перечней - см. статью 3 Федерального закона от 9 января 1996 года N 2-ФЗ. - Примечание изготовителя базы данных.\n\n____________________________________________________________________', metadata={'source': 'law.pdf'}), Document(page_content='_

### Create chromadb Vectorstore

In [10]:
from langchain_chroma import Chroma
from langchain_mistralai import MistralAIEmbeddings

# vectorstore.delete_collection("Zakon_recursive_text_chunking")
vectorstore = Chroma.from_documents(documents=chunked_document, embedding=MistralAIEmbeddings(), persist_directory="chroma_directory", collection_name="MistralAI_recursive_text_chunking")


/Users/daniilkoveh/Library/Mobile Documents/com~apple~CloudDocs/Langchain/env/lib/python3.12/site-packages/langchain_mistralai/embeddings.py:105: UserWarning: Could not download mistral tokenizer from Huggingface for calculating batch sizes. Set a Huggingface token via the HF_TOKEN environment variable to download the real tokenizer. Falling back to a dummy tokenizer that uses `len()`.
  warnings.warn(


In [11]:
retriever = vectorstore.as_retriever(k=4)

In [12]:
# check if the retriever works
query = "сколько дней у потребителя на обмен или возврат товара?"
docs = retriever.invoke(query)
docs

[Document(page_content='Комментарий к статье 20\n\nСтатья 21. Замена товара ненадлежащего качества\n\n1. В случае обнаружения потребителем недостатков товара и предъявления требования о его замене продавец (изготовитель, уполномоченная организация или уполномоченный индивидуальный предприниматель, импортер) обязан заменить такой товар в течение семи дней со дня предъявления указанного требования потребителем, а при необходимости дополнительной проверки качества такого товара продавцом (изготовителем, уполномоченной организацией или уполномоченным индивидуальным предпринимателем, импортером) - в течение двадцати дней со дня предъявления указанного требования. индивидуального предпринимателя, импортера) в момент предъявления требования отсутствует необходимый для замены товар, замена должна быть проведена в течение месяца со дня предъявления такого требования.', metadata={'source': 'law.pdf'}),
 Document(page_content='Система ГАРАНТ\n\n16/27\n\nЗакон РФ от 7 февраля 1992 г. N 2300-I "О з

# RAG

### Set up LLM

In [14]:
from langchain_core.messages import HumanMessage
# from langchain_openai import ChatOpenAI
# from langchain_mistralai import ChatMistralAI
from langchain_anthropic import ChatAnthropic

import os
from dotenv import load_dotenv
load_dotenv()

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

# llm = ChatMistralAI(temperature=0)

llm = ChatAnthropic(temperature=0,
                    model_name="claude-3-haiku-20240307", 
                    api_key=os.getenv('ANTHROPIC_API_KEY'))

Create a prompt template

In [15]:
#add the documents to the prompt
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

SYSTEM_TEMPLATE = """Ты – юрист. Ответь на вопрос исходя из текстов ниже. Если не знаешь: 'Я не знаю'. Четко и кратко и верно ответь. Вот тексты:

<context>
{context}
</context>
"""

question_answering_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", SYSTEM_TEMPLATE),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

document_chain = create_stuff_documents_chain(llm, question_answering_prompt)

Ask a question. question first will be indexed in the vectorstore and then top 4 results will be added in the prompt.

In [16]:
document_chain.invoke(
    {
        "context": docs,
        "messages": [
            HumanMessage(content="когда я могу вернуть товар обратно в магазин?")
        ],
    }
)

'Согласно статье 21 Закона РФ "О защите прав потребителей":\n\n1. Если потребитель обнаружил недостатки товара и предъявил требование о его замене, продавец (изготовитель, уполномоченная организация или индивидуальный предприниматель, импортер) обязан заменить такой товар в течение 7 дней со дня предъявления этого требования. \n\n2. Если для замены товара требуется более 7 дней, по требованию потребителя продавец (изготовитель, уполномоченная организация или индивидуальный предприниматель) в течение 3 дней должен предоставить потребителю во временное пользование на период замены аналогичный товар.\n\n3. Если необходима дополнительная проверка качества товара, замена должна быть проведена в течение 20 дней со дня предъявления требования.\n\n4. Если необходимого для замены товара нет в наличии, замена должна быть проведена в течение месяца со дня предъявления требования.\n\nТаким образом, в соответствии с законом, вы можете вернуть товар обратно в магазин в случае обнаружения его недоста